# Run inference on time to merge model trained previously


## What we did previously

In the previous [notebook](./03_model_training.ipynb) we trained machine learning models to classify a PR's `time_to_merge` into one of the 10 bins (or "classes"). We then deployed the model with the highest f1-score as a service using the model saved in s3.

## In this step

In this notebook, we are going to fetch the model that we previously trained, saved and stored in s3. We will send a payload to this model and see how it performs on the test data.
# Time to Merge Prediction Inference Service

In the previous notebook, we explored some basic machine learning models for predicting time to merge of a PR.

In [1]:
import os
import sys
import gzip
import json
import boto3
import datetime
import requests
from dotenv import load_dotenv, find_dotenv
from io import BytesIO

import joblib

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

load_dotenv(find_dotenv(), override=True)

import warnings
warnings.filterwarnings('ignore') 

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
class CephCommunication:
    """
    Class to establish communication with a ceph s3 bucket.
    It connects with the bucket and provides methods to read and write data in the parquet format.
    """

    def __init__(
        self, s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
    ):
        self.s3_endpoint_url = s3_endpoint_url
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.s3_resource = boto3.resource(
            "s3",
            endpoint_url=self.s3_endpoint_url,
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key,
        )
        self.bucket = s3_bucket
        ## Todo: Add try catch

    def upload_to_ceph(self, dataframe, s3_path, filename):
        """
        This helper function takes as input the data frame to be uploaded, and the output filename.
        It then saves the data frame in the defined ceph bucket.
        """
        parquet_buffer = BytesIO()
        dataframe.to_parquet(parquet_buffer)
        s3_obj = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        status = s3_obj.put(Body=parquet_buffer.getvalue())
        return status

    def read_from_ceph(self, s3_path, filename):
        """
        Helper function to read from ceph and see if the saved data is correct.
        """
        buffer = BytesIO()
        s3_object = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        s3_object.download_fileobj(buffer)
        df_temp = pd.read_parquet(buffer)
        return df_temp


def save_to_disk(dataframe, path, filename):
    """
    Helper function to save the dataframe
    as a parquet file to disk.
    """
    dataset_base_path = Path(path)
    dataset_base_path.mkdir(parents=True, exist_ok=True)
    dataframe.to_parquet(f"{path}/{filename}")
    return True

In [3]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,

ORG = os.getenv("GITHUB_ORG")
REPO = os.getenv("GITHUB_REPO")

## S3 bucket credentials
s3_endpoint_url = os.getenv("S3_ENDPOINT_URL")
s3_access_key = os.getenv("AWS_ACCESS_KEY_ID")
s3_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_bucket = os.getenv("S3_BUCKET")

s3_input_data_path = os.getenv("CEPH_BUCKET_PREFIX")

REMOTE = os.getenv("REMOTE")
RAW_DATA_PATH = os.path.join(
    s3_input_data_path, "srcopsmetrics/bot_knowledge", ORG, REPO, "PullRequest.json"
)

In [4]:
output = []
local_input_data_path = "../../../data/raw/GitHub/PullRequest.json.gz"
if REMOTE:
    print("getting dataset from ceph")
    s3 = boto3.resource(
        "s3",
        endpoint_url=s3_endpoint_url,
        aws_access_key_id=s3_access_key,
        aws_secret_access_key=s3_secret_key,
    )
    content = s3.Object(s3_bucket, RAW_DATA_PATH)
    file = content.get()["Body"].read().decode("utf-8")

    prs = json.loads(file)

    for pr in prs.splitlines():
        output.append(json.loads(pr))

else:
    print("getting dataset from local")
    with gzip.open(local_input_data_path, "r") as f:
        prs = json.loads(f.read().decode("utf-8"))


pr_df = pd.DataFrame(output)

getting dataset from ceph


In [5]:
# github pr dataset collected using thoth's mi-scheduler
pr_df.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Update all remaining SRE docs with mentions of...,These changes have been made to 2 documents (m...,XS,carlosGimeno,1642529077,1642621111,sesheta,1.642621e+09,sesheta,1,2,{'sesheta': 65},"{'857262082': {'author': 'HumairAK', 'words_co...","[approved, size/XS, lgtm]",[ee2a7c9f72f9b16aeb83bf7b140f080268a27483],"[dashboards/README.md, incident-management/con...",1.642621e+09,1.642621e+09,443
1,Fix operate-first.cloud links,Existing links give 404s,XS,david-martin,1641919743,1659120271,sesheta,NaN,None,1,1,{'sesheta': 346},"{'892466202': {'author': 'larsks', 'words_coun...","[size/XS, lifecycle/rotten, needs-ok-to-test]",[ac67e7fc600c0030a45f114329165666f714977c],[README.md],1.645711e+09,NaN,438
2,Move runbooks to apps repo.,Merge after: https://github.com/operate-first/...,L,HumairAK,1636043225,1636044247,sesheta,1.636044e+09,sesheta,1,4,{'sesheta': 65},"{'798001484': {'author': '4n4nd', 'words_count...","[approved, size/L, lgtm]",[450a5561904f20de4ff63c6265461b3f5eea5070],"[runbooks/TEMPLATE.md, runbooks/jupyterhub.md,...",1.636044e+09,1.636044e+09,420
3,fix link in README.md,fix link back to main page,XS,axelsimon,1635255508,1635266618,sesheta,1.635267e+09,sesheta,1,1,"{'sesheta': 182, 'axelsimon': 2}","{'789656199': {'author': 'HumairAK', 'words_co...","[approved, size/XS, lgtm, needs-ok-to-test]",[7d47a5f9e48a06466b627fde10f99f782bc4a7c8],[README.md],1.635266e+09,1.635266e+09,412
4,Update instructions to fix superset key issue.,Reflecting findings from [here](https://github...,XS,HumairAK,1634676624,1634677018,HumairAK,1.634677e+09,HumairAK,1,1,{'sesheta': 74},"{'783773951': {'author': '4n4nd', 'words_count...","[approved, size/XS]",[6c70460bd9634abeb53e5b7eec4965d10c2fa24a],[runbooks/superset.md],1.634677e+09,1.634677e+09,409


In [6]:
interval = (pr_df["merged_at"] - pr_df["created_at"]).astype("float")
interval = interval.dropna()
interval

0       92034.0
2        1022.0
3       11110.0
4         394.0
5      329836.0
6         299.0
7       44256.0
8        3446.0
10      11614.0
11        169.0
12      14923.0
13     164048.0
14       6546.0
15        888.0
16     247656.0
17        132.0
18        294.0
19       8735.0
20       3974.0
21     151358.0
22     137488.0
23     317515.0
24     236050.0
25      78160.0
26     348768.0
27        874.0
28        449.0
29      24669.0
30        426.0
31       1388.0
32     128183.0
33      56415.0
34       3301.0
35        102.0
36      68192.0
37      72375.0
38    2919683.0
39      68319.0
41    1280968.0
42     406821.0
43      65538.0
dtype: float64

In [7]:
n_buckets = 10

quantiles = interval.quantile(q=np.arange(0, 1 + 1e-100, 1 / n_buckets))
quantiles

0.0       102.0
0.1       299.0
0.2       874.0
0.3      3301.0
0.4      8735.0
0.5     24669.0
0.6     68192.0
0.7     92034.0
0.8    164048.0
0.9    329836.0
dtype: float64

In [8]:
quantiles / 3600

0.0     0.028333
0.1     0.083056
0.2     0.242778
0.3     0.916944
0.4     2.426389
0.5     6.852500
0.6    18.942222
0.7    25.565000
0.8    45.568889
0.9    91.621111
dtype: float64

In [9]:
time_intervals = quantiles / 3600

In [10]:
# remove PRs from train/test which are still open
pr_df = pr_df[pr_df["closed_at"].notna()]
pr_df = pr_df[pr_df["merged_at"].notna()]

In [11]:
pr_df["created_at"] = pr_df["created_at"].apply(
    lambda x: int(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["closed_at"] = pr_df["closed_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["merged_at"] = pr_df["merged_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)

In [12]:
TEST_DATA_PATH = os.path.join(s3_input_data_path, ORG, REPO, "test-data")

# read processed and split data created for train/test in the model training notebook
if REMOTE:
    cc = CephCommunication(s3_endpoint_url, s3_access_key, s3_secret_key, s3_bucket)
    X_test = cc.read_from_ceph(TEST_DATA_PATH, "X_test.parquet")
    y_test = cc.read_from_ceph(TEST_DATA_PATH, "y_test.parquet")

else:
    print(
        "The X_test.parquet and y_test.parquet files are not included in the github repo."
    )
    print(
        "Please set REMOTE=1 in the .env file and read this data from the S3 bucket instead."
    )

In [13]:
X_test

,size,created_at_day,created_at_month,created_at_weekday,created_at_hour,changed_files_number,body_size,commits_number,filetype_.md,filetype_.json,...,title_wordcount_sre,title_wordcount_superset,title_wordcount_template,title_wordcount_threads,title_wordcount_update,title_wordcount_upgrade,title_wordcount_usage,title_wordcount_website,title_wordcount_wip,title_wordcount_zero
26,1.0,5.0,3.0,4.0,17.0,3.0,3.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
15,0.0,20.0,4.0,1.0,16.0,1.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,17.0,6.0,3.0,9.0,1.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
27,1.0,16.0,2.0,1.0,18.0,1.0,5.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5,2.0,1.0,10.0,4.0,21.0,2.0,11.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
43,2.0,6.0,10.0,1.0,20.0,1.0,3.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,3.0,29.0,3.0,0.0,20.0,1.0,6.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,2.0,7.0,10.0,2.0,20.0,1.0,9.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,1.0,1.0,2.0,0.0,20.0,1.0,3.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_test

,ttm_class
26,9
15,2
10,4
27,2
5,9
43,5
21,7
42,9
31,2


In [15]:
sample_payload = pr_df.reindex(X_test.index)

In [16]:
sample_payload

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
26,Update links to point to new cluster routes.,Part of: https://github.com/operate-first/SRE/...,S,HumairAK,1,1.615312,HumairAK,1.615312,HumairAK,1,3,"{'sesheta': 85, 'HumairAK': 1}","{'606187139': {'author': 'tumido', 'words_coun...","[do-not-merge/hold, size/S, lgtm]",[4604438d112e362edfb7fbc42e9ad561640587ea],"[README.md, dashboards/README.md, incident-man...",1.615202e+09,NaN,114
15,Fix grafana links in JH runbook,Closes https://github.com/operate-first/SRE/is...,XS,4n4nd,1,1.618935,sesheta,1.618935,sesheta,1,1,"{'hemajv': 7, 'sesheta': 65}","{'640169620': {'author': 'hemajv', 'words_coun...","[approved, size/XS]",[b9f2cf436d04be832825a6e9deff3d44b34b2977],[runbooks/jupyterhub.md],1.618936e+09,1.618936e+09,262
10,fix: Update JH dashboard links,Resolves: https://github.com/operate-first/ope...,XS,tumido,1,1.623933,sesheta,1.623933,sesheta,1,1,{'sesheta': 65},"{'686200348': {'author': '4n4nd', 'words_count...","[approved, size/XS, lgtm]",[22412286c34e4d7e0074effff140f635e35a9914],[runbooks/jupyterhub.md],1.623930e+09,1.623933e+09,344
27,Update JH usage dashboard,Updating the [JH usage dashboard](https://graf...,S,hemajv,1,1.613501,sesheta,1.613501,sesheta,1,1,"{'anishasthana': 1, 'HumairAK': 1, 'sesheta': 65}",{},"[approved, size/S, lgtm]",[3d5854874530ef561bea1e23df36fa2e5d70afe2],[dashboards/jupyterhub-usage.json],NaN,NaN,65
5,update prose for website,Updating prose for website to make more beginn...,M,isabelizimm,1,1.633452,hemajv,1.633452,hemajv,1,2,"{'isabelizimm': 3, 'schwesig': 12, 'sesheta': 68}","{'771387664': {'author': 'schwesig', 'words_co...","[size/M, approved]",[211ad279933fe6db8e6a7437af7efc240e2a574f],"[README.md, sli-slo/tools.md]",1.633438e+09,1.633449e+09,397
43,Add JH slis,this closes operate-first/support#965,M,hemajv,1,1.602080,hemajv,1.602080,hemajv,1,1,{'sesheta': 85},"{'503447592': {'author': 'anishasthana', 'word...",[size/M],[cbccc50465cb126884d82f5f1a0c5e439d2570f1],[sli-slo/jupyterhub.md],1.602030e+09,1.602030e+09,6
21,Add Kafka runbook,Runbook for Kafka\r\n\r\n(this addresses opera...,L,hemajv,1,1.617202,sesheta,1.617202,sesheta,1,1,"{'HumairAK': 26, 'maulikjs': 7, 'sesheta': 65}","{'624040450': {'author': 'tumido', 'words_coun...","[approved, size/L, lgtm]",[ad0d36214be2492eb306d0cf47af25d4e654c018],[runbooks/kafka.md],1.617097e+09,NaN,233
42,Add runbook template for jupyterhub,Adding a runbook template for jupyterhub\r\n\r...,M,hemajv,1,1.602510,hemajv,1.602510,hemajv,1,1,{'sesheta': 82},"{'504314683': {'author': 'anishasthana', 'word...",[size/M],[6f80a7bd3ff99a4b232fac2c5f08be76be558d1c],[runbooks/jupyterhub.md],1.602110e+09,1.602110e+09,7
31,Update README,this closes operate-first/support#933,S,hemajv,1,1.612213,sesheta,1.612213,sesheta,1,1,"{'sesheta': 77, 'HumairAK': 10}","{'580750754': {'author': 'anishasthana', 'word...","[approved, size/S]",[9f42ba5a08f01f32160a2487de3e61cf4e9aa5a0],[README.md],1.612212e+09,1.612212e+09,53


In [17]:
sample_payload.dtypes

title                    object
body                     object
size                     object
created_by               object
created_at                int64
closed_at               float64
closed_by                object
merged_at               float64
merged_by                object
commits_number            int64
changed_files_number      int64
interactions             object
reviews                  object
labels                   object
commits                  object
changed_files            object
first_review_at         float64
first_approve_at        float64
id                       object
dtype: object

In [18]:
## read model
MODEL_KEY = os.path.join(s3_input_data_path, ORG, REPO, "ttm-model")
MODEL_FILENAME = "model.joblib"


s3_resource = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

buffer = BytesIO()
s3_object = s3_resource.Object(s3_bucket, f"{MODEL_KEY}/{MODEL_FILENAME}")
s3_object.download_fileobj(buffer)
model = joblib.load(buffer)
model

Pipeline(steps=[('extract_features',
                 ColumnTransformer(transformers=[('pr_size',
                                                  OrdinalEncoder(categories=[['XS',
                                                                              'S',
                                                                              'M',
                                                                              'L',
                                                                              'XL',
                                                                              'XXL']]),
                                                  ['size']),
                                                 ('created_at_details',
                                                  DateTimeDetailsTransformer(),
                                                  ['created_at']),
                                                 ('n_changed_files',
                                                  NumChangedFilesTransformer(),
                                                  ['changed_files_number']),
                                                 ('body_size',
                                                  StringLenTransformer(),
                                                  ['body']),
                                                 ('n_commits', 'p...
                                                  TitleWordCountTransformer(words=['add',
                                                                                   'already',
                                                                                   'apps',
                                                                                   'availability',
                                                                                   'boilerplate',
                                                                                   'chore',
                                                                                   'ci',
                                                                                   'cloud',
                                                                                   'cluster',
                                                                                   'comprehensive',
                                                                                   'creating',
                                                                                   'dashboard',
                                                                                   'dashboards',
                                                                                   'defined',
                                                                                   'dir',
                                                                                   'discuss',
                                                                                   'docs',
                                                                                   'edits',
                                                                                   'errors',
                                                                                   'fairly',
                                                                                   'feature',
                                                                                   'files',
                                                                                   'first',
                                                                                   'fix',
                                                                                   'flooded',
                                                                                   'idh',
                                                                                   'incident',
                                                                                   'ipolonsk',
        

In [19]:
# Test model on the dataset
preds = model.predict(sample_payload)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         0
           4       0.33      1.00      0.50         1
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       1.00      0.33      0.50         3

    accuracy                           0.22         9
   macro avg       0.17      0.17      0.12         9
weighted avg       0.37      0.22      0.22         9



# Conclusion

This notebook fetches the saved model from s3 and sends a payload to see how the model is performing on this new data. Additionally, we see that the evaluation scores in the classification report match the ones we saw in the training notebook. So, great, looks like our model are working as expected, and are ready to predict some times to merge for GitHub PRs! 